In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install mlflow

In [ ]:
import digitalhub as dh

# Create the project 

In [ ]:
PROJECT = "demo-object-detection"
project = dh.get_or_create_project(PROJECT)

## Create the training function

In [ ]:
from digitalhub_runtime_python import handler
from digitalhub_ml.entities.utils import from_mlflow_run
import mlflow

from ultralytics import YOLO

mlflow.autolog(log_datasets=True)
with mlflow.start_run():
    # Load and train your YOLOv8 model
    model = YOLO('yolov8n.pt')
    results = model.train(data='coco8.yaml', epochs=100, imgsz=640)

    # Log parameters, metrics, and model
    mlflow.log_params({"epochs": 100, "model": "yolov8n.pt", "data": "coco8.yaml"})
    #mlflow.log_metrics({"mAP_0.5": results.map50, "mAP_0.5:0.95": results.map})
    mlflow.pytorch.log_model(model, "model")

    # End the run
    mlflow.end_run()

# utility to map mlflow run artifacts to model metadata
run_id = mlflow.last_active_run().info.run_id
model_params = from_mlflow_run(run_id)


In [ ]:
%%writefile functions/train-model.py

from digitalhub_runtime_python import handler
from digitalhub_ml.entities.utils import from_mlflow_run
import mlflow

from ultralytics import YOLO

@handler()
def train(project):
    mlflow.autolog(log_datasets=True)
    with mlflow.start_run():
        # Load and train your YOLOv8 model
        model = YOLO('yolov8n.pt')
        results = model.train(data='coco8.yaml', epochs=100, imgsz=640)
    
        # Log parameters, metrics, and model
        mlflow.log_params({"epochs": 100, "model": "yolov8n.pt", "data": "coco8.yaml"})
        mlflow.pytorch.log_model(model, "model")
    
        # End the run
        mlflow.end_run()

    # utility to map mlflow run artifacts to model metadata
    run_id = mlflow.last_active_run().info.run_id
    model_params = from_mlflow_run(run_id)

    project.log_model(
        name="model-mlflow-yolo",
        kind="mlflow",
        **model_params
)

In [ ]:
# Define the training function inside the project
train_fn = project.new_function(
                         name="train",
                         kind="python",
                         python_version="PYTHON3_9",
                         source={"source": "functions/train-model.py", "handler": "train"},
                         requirements=["ultralytics==8.2.82", "mlflow==2.15.1"])

In [ ]:
train_run = train_fn.run(action="job", local_execution=True)

# Simple training version

In [ ]:

from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="coco8.yaml", epochs=100, imgsz=640)

In [ ]:
results = model(["000000000061.jpg"])

In [ ]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk